In [1]:
from embit.psbt import PSBT
from binascii import a2b_base64, b2a_base64
from io import BytesIO
from embit import ec

from embit import bip32, bip39
from embit.psbt import PSBT
from embit.descriptor import Descriptor
from binascii import hexlify






In [2]:
def detect_psbt_sig(psbt_str):
    "parase the PSBT for the 'partial_sigs' method"
    psbt = PSBT.from_string(psbt_str)
    if len(psbt.inputs[0].partial_sigs) == 0:
        raise ValueError("No Signatures Detected")
    if len(psbt.inputs[0].partial_sigs) > 1:
        raise ValueError("More than one Signature Detected. Not Intended Behavior")
    
    #WARNING:
    #This is assuming provided signed pubkey is honest
    #Add validation that the signature belongs to the pubkey
    signed_pubkey = list(psbt.inputs[0].partial_sigs.keys())[0]
    
    return signed_pubkey.to_string()

#Grabs Public Key(s) used in transaction
#need to update to grab index and branch dynamically
def get_pubkeys(descriptor,idx=0,branch=0):
    keymap = {}
    
    xpubs = []
    for i in range(len(descriptor.derive(idx=0, branch_index=0).keys)):
        pubkeys = []
        xpub = descriptor.keys[i].key.to_string()
        xpubs.append(xpub)
        
        pubkey = descriptor.derive(idx, branch_index=branch).keys[i].get_public_key().to_string()
        #print(pubkey)
        pubkeys.append(pubkey)
        keymap[pubkey]=xpub
    return keymap

def scan_verify(descriptor,signed_pubkey):
    for i in range(100):
        branch = 0
        some_pubkeys = get_pubkeys(descriptor,idx=i,branch=branch)
        #print(list(some_pubkeys))
        if signed_pubkey in list(get_pubkeys(descriptor,idx=i,branch=0).keys()):
            print(f"Found the match! {signed_pubkey} is in {some_pubkeys}")
        
        branch = 1
        some_pubkeys = list(get_pubkeys(descriptor,idx=i,branch=branch).keys())
        #print(list(some_pubkeys))
        if signed_pubkey in list(some_pubkeys):
            print(f"Found the match! {signed_pubkey} is in {some_pubkeys} at index {i} and branch {branch}")
            return i,branch

In [3]:
#opens PSBT
f = open("testnet-onesig.psbt", "rb")
string = f.read()

#encodes to PSBT format
onesig_psbt_byte = b2a_base64(string)


#extracts signed pubkey from PSBT
signed_pubkey = detect_psbt_sig(onesig_psbt_byte.decode("utf-8"))

#testnet descriptor
desc = Descriptor.from_string("wsh(sortedmulti(2,tpubDE6RguwrqWuvELAd2BVbP4xi9DYTxWYsZ8Pu1KbU4ttbcmhw7i2MM7CZTubYa48Mfc5garWZ6GtiS2oAVFDYhai8bGG7GeagP6RnV7gbHBf/{0,1}/*,tpubDFKHqqaYCjf8Uff3X7YpEsf1uQ2LrWo7bBw5qT87aJqem98ZkrBE1XPJFzvHpgz2ZKCPP6a9agULkymCRtcpKtKurMfJN8BMqRQLYSUZ6N4/{0,1}/*,tpubDFi2WWwmGmwdGNdZ97o9dSFiQunzadfK57Qwbpcx73dEBtUaPSna65kUS92mS8xvWFHv7fUwpj5LibRBNGGYFak7ShretiB5cv5GXUDGzfZ/{0,1}/*))")

#extracts the branch and index the key was signed from
i, branch = scan_verify(desc,signed_pubkey)

#generates the keymap using the targeted branch and index
test_keymap = get_pubkeys(desc,idx=i,branch=branch)

#returns the xpub the signature corresponds to.
test_keymap[signed_pubkey]

Found the match! 03a41ba2ca720548533be5b7046176bc9248e5347274a2c2ceee5a21d66b768d35 is in ['03a41ba2ca720548533be5b7046176bc9248e5347274a2c2ceee5a21d66b768d35', '028f72df751aec4e3de54dbe2f8ec75b534aa0d404bc6565aceb183f492117e507', '02f65f38f3de56805aff7f0d35232f4cedc64f7e766a2f9e9383afbb1db6f5db34'] at index 0 and branch 1


'tpubDE6RguwrqWuvELAd2BVbP4xi9DYTxWYsZ8Pu1KbU4ttbcmhw7i2MM7CZTubYa48Mfc5garWZ6GtiS2oAVFDYhai8bGG7GeagP6RnV7gbHBf'